# 🛍️ Global Fashion Sales Performance Report
This notebook presents an analytical overview of the **Global Fashion dataset**, designed for buisnes insights

The analysis follows a structured business intelligence flow used in Power BI-style dashboards to uncover key insights on:
- Revenue performance and growth drivers  
- Product profitability and category trends  
- Return patterns and operational inefficiencies  
- Customer segmentation and value identification  
- Payment method analysis and sales seasonality  
- Discount impact and marketing effectiveness  

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [6]:
# Path to folder
path = "/content/"

customers = pd.read_csv(path + "customers_clean.csv", low_memory=False)
products = pd.read_csv(path + "products_clean.csv")
transactions = pd.read_csv(path + "transactions_clean.csv")
stores = pd.read_csv(path + "stores_clean.csv")
employees = pd.read_csv(path + "employees_clean.csv")
discounts = pd.read_csv(path + "discounts_clean.csv")

customers.shape, products.shape, transactions.shape, stores.shape, employees.shape, discounts.shape

((1643306, 9), (14950, 13), (6284272, 20), (35, 8), (403, 4), (204, 6))

In [7]:
# Convert dates
transactions['Date'] = pd.to_datetime(transactions['Date'])
# Create return flag
transactions['is_return'] = (transactions['Line_Total'] < 0).astype(int)

In [11]:
# EXECUTIVE REVENUE OVERVIEW
print("\n" + "=" * 80)
print("EXECUTIVE REVENUE OVERVIEW")
print("=" * 80)

# Calculate key metrics
gross_revenue = transactions[transactions['is_return'] == 0]['Line_Total'].sum()
returns_loss = transactions[transactions['is_return'] == 1]['Line_Total'].sum()
net_revenue = transactions['Line_Total'].sum()
return_rate = abs(returns_loss / gross_revenue) * 100

print(f"\n💰 REVENUE SUMMARY:")
print(f"   Gross Revenue: ${gross_revenue:,.2f}")
print(f"   Returns (Loss): ${returns_loss:,.2f}")
print(f"   Net Revenue: ${net_revenue:,.2f}")
print(f"   Return Rate: {return_rate:.2f}%")

# Time period
date_range_days = (transactions['Date'].max() - transactions['Date'].min()).days
print(f"\n📅 TIME PERIOD:")
print(f"   Start Date: {transactions['Date'].min().date()}")
print(f"   End Date: {transactions['Date'].max().date()}")
print(f"   Total Days: {date_range_days:,}")
print(f"   Avg Daily Revenue: ${net_revenue / date_range_days:,.2f}")

# Transaction metrics
total_orders = transactions['Invoice_ID'].nunique()
avg_order_value = transactions.groupby('Invoice_ID')['Invoice_Total'].first().mean()
total_units_sold = transactions[transactions['is_return'] == 0]['Quantity'].sum()

print(f"\n📦 TRANSACTION METRICS:")
print(f"   Total Orders: {total_orders:,}")
print(f"   Average Order Value: ${avg_order_value:,.2f}")
print(f"   Total Units Sold: {total_units_sold:,.0f}")
print(f"   Avg Items per Order: {total_units_sold / total_orders:.1f}")

fig = go.Figure()

fig.add_trace(go.Bar(
    name='Gross Revenue',
    x=['Revenue Breakdown'],
    y=[gross_revenue],
    marker_color='lightblue',
    text=[f'${gross_revenue/1e6:.1f}M'],
    textposition='inside'
))

fig.add_trace(go.Bar(
    name='Returns',
    x=['Revenue Breakdown'],
    y=[abs(returns_loss)],
    marker_color='salmon',
    text=[f'-${abs(returns_loss)/1e6:.1f}M'],
    textposition='inside'
))

fig.add_trace(go.Bar(
    name='Net Revenue',
    x=['Revenue Breakdown'],
    y=[net_revenue],
    marker_color='lightgreen',
    text=[f'${net_revenue/1e6:.1f}M'],
    textposition='inside'
))

fig.update_layout(
    title='Revenue Overview: Gross vs Net',
    yaxis_title='Revenue ($)',
    barmode='group',
    height=400
)

fig.show()


EXECUTIVE REVENUE OVERVIEW

💰 REVENUE SUMMARY:
   Gross Revenue: $807,803,874.00
   Returns (Loss): $-45,324,477.54
   Net Revenue: $762,479,396.46
   Return Rate: 5.61%

📅 TIME PERIOD:
   Start Date: 2023-01-01
   End Date: 2025-02-22
   Total Days: 783
   Avg Daily Revenue: $973,792.33

📦 TRANSACTION METRICS:
   Total Orders: 4,448,433
   Average Order Value: $172.26
   Total Units Sold: 6,542,869
   Avg Items per Order: 1.5


# A. REVENUE ANALYSIS

In [8]:
# --- A1. Revenue by Country ---
print("\n📍 A1. REVENUE BY COUNTRY")

trans_country = transactions.merge(customers[['Customer_ID', 'Country']], on='Customer_ID')
revenue_by_country = trans_country.groupby('Country')['Line_Total'].sum().sort_values(ascending=False)

print("\nRevenue by Country:")
for country, revenue in revenue_by_country.items():
    print(f"   {country:15s}: ${revenue:>15,.2f}")

# Visualization
fig = go.Figure(data=[
    go.Bar(
        x=revenue_by_country.index,
        y=revenue_by_country.values,
        marker_color='lightblue',
        text=[f'${v/1e6:.1f}M' for v in revenue_by_country.values],
        textposition='outside'
    )
])
fig.update_layout(
    title='Revenue by Country',
    xaxis_title='Country',
    yaxis_title='Revenue ($)',
    height=400
)
fig.show()

# --- A2. Revenue by Category ---
print("\n📦 A2. REVENUE BY CATEGORY")

trans_category = transactions.merge(products[['Product_ID', 'Category']], on='Product_ID')
revenue_by_category = trans_category.groupby('Category')['Line_Total'].sum().sort_values(ascending=False)

print("\nRevenue by Category:")
for category, revenue in revenue_by_category.items():
    print(f"   {category:10s}: ${revenue:>15,.2f}")

# Visualization
fig = go.Figure(data=[
    go.Pie(
        labels=revenue_by_category.index,
        values=revenue_by_category.values,
        hole=0.4,
        textinfo='label+percent',
        textfont_size=14
    )
])
fig.update_layout(title='Revenue Distribution by Category', height=400)
fig.show()

# --- A3. Monthly Revenue Trend ---
print("\n📅 A3. MONTHLY REVENUE TREND")

transactions['Date'] = pd.to_datetime(transactions['Date'])
transactions['YearMonth'] = transactions['Date'].dt.to_period('M')

monthly_revenue = transactions.groupby('YearMonth')['Line_Total'].sum().reset_index()
monthly_revenue['YearMonth'] = monthly_revenue['YearMonth'].astype(str)

print("\nLast 12 Months Revenue:")
for _, row in monthly_revenue.tail(12).iterrows():
    print(f"   {row['YearMonth']}: ${row['Line_Total']:>12,.2f}")

# Visualization
fig = go.Figure(data=[
    go.Scatter(
        x=monthly_revenue['YearMonth'],
        y=monthly_revenue['Line_Total'],
        mode='lines+markers',
        line=dict(color='blue', width=2),
        marker=dict(size=8)
    )
])
fig.update_layout(
    title='Monthly Revenue Trend',
    xaxis_title='Month',
    yaxis_title='Revenue ($)',
    height=400
)
fig.update_xaxes(tickangle=45)
fig.show()

# --- A4. Top 10 Products by Revenue ---
print("\n🏆 A4. TOP 10 PRODUCTS BY REVENUE")

product_revenue = transactions[transactions['is_return'] == 0].groupby('Product_ID')['Line_Total'].sum().sort_values(ascending=False).head(10)
product_names = products.set_index('Product_ID')['Description_EN']

print("\nTop 10 Products:")
for i, (prod_id, revenue) in enumerate(product_revenue.items(), 1):
    name = product_names.get(prod_id, 'Unknown')[:50]
    print(f"   {i:2d}. {name:50s} ${revenue:>12,.2f}")

# --- A5. Revenue by Store ---
print("\n🏬 A5. REVENUE BY STORE")

revenue_by_store = transactions.groupby('Store_ID')['Line_Total'].sum().sort_values(ascending=False).head(10)
store_names = stores.set_index('Store_ID')['Store_Name']

print("\nTop 10 Stores by Revenue:")
for i, (store_id, revenue) in enumerate(revenue_by_store.items(), 1):
    name = store_names.get(store_id, 'Unknown')
    print(f"   {i:2d}. {name:20s} ${revenue:>15,.2f}")

# Visualization
store_data = pd.DataFrame({
    'Store': [store_names.get(sid, 'Unknown') for sid in revenue_by_store.index],
    'Revenue': revenue_by_store.values
})

fig = go.Figure(data=[
    go.Bar(
        x=store_data['Store'],
        y=store_data['Revenue'],
        marker_color='teal',
        text=[f'${v/1e6:.1f}M' for v in store_data['Revenue']],
        textposition='outside'
    )
])
fig.update_layout(
    title='Top 10 Stores by Revenue',
    xaxis_title='Store',
    yaxis_title='Revenue ($)',
    height=400
)
fig.update_xaxes(tickangle=45)
fig.show()

# --- A6. Discount Campaign Revenue ---
print("\n🏷️ A6. DISCOUNT CAMPAIGN REVENUE")

trans_positive = transactions[transactions['is_return'] == 0].copy()
trans_positive['has_discount'] = (trans_positive['Discount'] > 0).astype(int)

campaign_revenue = trans_positive[trans_positive['has_discount'] == 1]['Line_Total'].sum()
regular_revenue = trans_positive[trans_positive['has_discount'] == 0]['Line_Total'].sum()

print(f"\n   Campaign Revenue:  ${campaign_revenue:>15,.2f}")
print(f"   Regular Revenue:   ${regular_revenue:>15,.2f}")
print(f"   Campaign Share:    {campaign_revenue/(campaign_revenue+regular_revenue)*100:>14.2f}%")

# Visualization
fig = go.Figure(data=[
    go.Pie(
        labels=['Campaign Sales', 'Regular Sales'],
        values=[campaign_revenue, regular_revenue],
        hole=0.4,
        marker_colors=['#e74c3c', '#3498db']
    )
])
fig.update_layout(title='Revenue: Campaign vs Regular Sales', height=400)
fig.show()


📍 A1. REVENUE BY COUNTRY

Revenue by Country:
   中国             : $ 577,871,033.66
   United States  : $  71,041,666.44
   Deutschland    : $  28,528,341.79
   France         : $  23,456,469.97
   España         : $  23,235,818.40
   Portugal       : $  19,446,468.34
   United Kingdom : $  18,899,597.86



📦 A2. REVENUE BY CATEGORY

Revenue by Category:
   Feminine  : $ 352,710,264.05
   Masculine : $ 342,006,832.05
   Children  : $  67,762,300.36



📅 A3. MONTHLY REVENUE TREND

Last 12 Months Revenue:
   2024-03: $37,889,952.08
   2024-04: $29,402,635.23
   2024-05: $30,339,108.77
   2024-06: $20,794,610.78
   2024-07: $21,801,680.05
   2024-08: $18,294,159.50
   2024-09: $44,278,783.89
   2024-10: $42,367,608.23
   2024-11: $30,257,382.70
   2024-12: $65,741,264.40
   2025-01: $17,310,289.24
   2025-02: $9,467,703.09



🏆 A4. TOP 10 PRODUCTS BY REVENUE

Top 10 Products:
    1. Men'S Blazer With Clean Style And Contemporary Cut $  288,375.55
    2. Male Fact With Built -In Pocket Detail             $  288,052.95
    3. Male Fact With Slim Fit Style And Built -In Belt D $  286,035.20
    4. Male Fact With Flanelado Fabric For Cold Climate   $  282,903.00
    5. Male Fact Of Casual Style Striped Twill            $  279,263.70
    6. Male Fact With Detail Of Leather Appliques         $  275,090.85
    7. Men'S Blazer Of Light Fabric With Cut In Thin Stri $  273,758.20
    8. Men'S Blazer With Zipper Details On The Sleeves    $  272,916.00
    9. Men'S Blazer With Embroidery Details In The Lapel  $  268,167.05
   10. Male Fact Slim Fit With Combined Fabric And Standa $  267,282.90

🏬 A5. REVENUE BY STORE

Top 10 Stores by Revenue:
    1. Store 上海             $ 181,009,484.33
    2. Store 北京             $ 137,549,118.66
    3. Store 广州             $  94,788,815.12
    4. Store 深圳             $  85,922,538.


🏷️ A6. DISCOUNT CAMPAIGN REVENUE

   Campaign Revenue:  $ 153,576,542.75
   Regular Revenue:   $ 654,227,331.25
   Campaign Share:             19.01%


# B. RETURN ANALYSIS

In [6]:
# B. RETURN ANALYSIS
# =====================================================================
print("\n" + "="*80)
print("B. RETURN ANALYSIS")
print("="*80)

# Calculate gross and returns
gross_revenue = transactions[transactions['is_return'] == 0]['Line_Total'].sum()
return_loss = transactions[transactions['is_return'] == 1]['Line_Total'].sum()
net_revenue = transactions['Line_Total'].sum()
overall_return_rate = abs(return_loss / gross_revenue * 100)

print(f"\n💰 OVERALL RETURN METRICS:")
print(f"   Gross Revenue:  ${gross_revenue:>15,.2f}")
print(f"   Return Loss:    ${return_loss:>15,.2f}")
print(f"   Net Revenue:    ${net_revenue:>15,.2f}")
print(f"   Return Rate:    {overall_return_rate:>14.2f}%")

# --- B1. Return Rate by Category ---
print("\n📦 B1. RETURN RATE BY CATEGORY")

category_gross = trans_category[trans_category['is_return'] == 0].groupby('Category')['Line_Total'].sum()
category_returns = trans_category[trans_category['is_return'] == 1].groupby('Category')['Line_Total'].sum()
category_return_rate = (abs(category_returns) / category_gross * 100).round(2)

print("\nReturn Rate by Category:")
for category, rate in category_return_rate.items():
    print(f"   {category:10s}: {rate:>6.2f}%")

# Visualization
fig = go.Figure(data=[
    go.Bar(
        x=category_return_rate.index,
        y=category_return_rate.values,
        marker_color='coral',
        text=[f'{v:.2f}%' for v in category_return_rate.values],
        textposition='outside'
    )
])
fig.update_layout(
    title='Return Rate by Category',
    xaxis_title='Category',
    yaxis_title='Return Rate (%)',
    height=400
)
fig.show()

# --- B2. Return Rate by Country ---
print("\n🌍 B2. RETURN RATE BY COUNTRY")

country_gross = trans_country[trans_country['is_return'] == 0].groupby('Country')['Line_Total'].sum()
country_returns = trans_country[trans_country['is_return'] == 1].groupby('Country')['Line_Total'].sum()
country_return_rate = (abs(country_returns) / country_gross * 100).round(2).sort_values(ascending=False)

print("\nReturn Rate by Country:")
for country, rate in country_return_rate.items():
    print(f"   {country:15s}: {rate:>6.2f}%")

# Visualization
fig = go.Figure(data=[
    go.Bar(
        x=country_return_rate.index,
        y=country_return_rate.values,
        marker_color='salmon',
        text=[f'{v:.2f}%' for v in country_return_rate.values],
        textposition='outside'
    )
])
fig.update_layout(
    title='Return Rate by Country',
    xaxis_title='Country',
    yaxis_title='Return Rate (%)',
    height=400
)
fig.show()

# --- B3. Return Loss by Category ---
print("\n💸 B3. RETURN LOSS BY CATEGORY")

print("\nReturn Loss by Category:")
for category, loss in category_returns.items():
    print(f"   {category:10s}: ${abs(loss):>15,.2f}")

# --- B4. Top 10 Products by Net Revenue ---
print("\n🏆 B4. TOP 10 PRODUCTS BY NET REVENUE")

product_net_revenue = transactions.groupby('Product_ID')['Line_Total'].sum().sort_values(ascending=False).head(10)

print("\nTop 10 Products by Net Revenue:")
for i, (prod_id, revenue) in enumerate(product_net_revenue.items(), 1):
    name = product_names.get(prod_id, 'Unknown')[:50]
    print(f"   {i:2d}. {name:50s} ${revenue:>12,.2f}")

# --- B5. Return Rate by Store ---
print("\n🏬 B5. RETURN RATE BY STORE")

store_gross = transactions[transactions['is_return'] == 0].groupby('Store_ID')['Line_Total'].sum()
store_returns = transactions[transactions['is_return'] == 1].groupby('Store_ID')['Line_Total'].sum()
store_return_rate = (abs(store_returns) / store_gross * 100).round(2).sort_values(ascending=False)

# Get store names
store_return_data = pd.DataFrame({
    'Store_ID': store_return_rate.index,
    'Return_Rate': store_return_rate.values
})
store_return_data = store_return_data.merge(stores[['Store_ID', 'Store_Name', 'Country']],
                                            left_on='Store_ID', right_on='Store_ID')

print("\nTop 10 Stores with Highest Return Rate:")
for i, (_, row) in enumerate(store_return_data.head(10).iterrows(), 1):
    print(f"   {i:2d}. {row['Store_Name']:20s} ({row['Country']:15s}): {row['Return_Rate']:>6.2f}%")

print("\nTop 10 Stores with Lowest Return Rate:")
for i, (_, row) in enumerate(store_return_data.tail(10).sort_values('Return_Rate').iterrows(), 1):
    print(f"   {i:2d}. {row['Store_Name']:20s} ({row['Country']:15s}): {row['Return_Rate']:>6.2f}%")

# Visualization
fig = go.Figure()

# Top 10 highest
top_10_highest = store_return_data.head(10)
fig.add_trace(go.Bar(
    name='Highest Return Rate',
    x=top_10_highest['Store_Name'],
    y=top_10_highest['Return_Rate'],
    marker_color='salmon',
    text=[f'{v:.2f}%' for v in top_10_highest['Return_Rate']],
    textposition='outside'
))

fig.update_layout(
    title='Top 10 Stores by Return Rate',
    xaxis_title='Store',
    yaxis_title='Return Rate (%)',
    height=400
)
fig.update_xaxes(tickangle=45)
fig.show()

# --- B6. Return Correlation Summary ---
print("\n📊 B6. RETURN CORRELATION SUMMARY")

# Best and worst stores
best_store = store_return_data.iloc[-1]
worst_store = store_return_data.iloc[0]

print(f"\n✅ BEST STORE (Lowest Returns):")
print(f"   Store: {best_store['Store_Name']}")
print(f"   Country: {best_store['Country']}")
print(f"   Return Rate: {best_store['Return_Rate']:.2f}%")

print(f"\n❌ WORST STORE (Highest Returns):")
print(f"   Store: {worst_store['Store_Name']}")
print(f"   Country: {worst_store['Country']}")
print(f"   Return Rate: {worst_store['Return_Rate']:.2f}%")

# Overall stats
print(f"\n📈 RETURN RATE STATISTICS:")
print(f"   Average Store Return Rate: {store_return_rate.mean():.2f}%")
print(f"   Median Store Return Rate:  {store_return_rate.median():.2f}%")
print(f"   Std Deviation:             {store_return_rate.std():.2f}%")
print(f"   Range:                     {store_return_rate.min():.2f}% - {store_return_rate.max():.2f}%")

# --- B7. Products with Lowest/Highest Returns ---
print("\n📦 B7. PRODUCTS WITH LOWEST/HIGHEST RETURNS")

# Calculate return rate per product
product_gross = transactions[transactions['is_return'] == 0].groupby('Product_ID')['Line_Total'].sum()
product_returns = transactions[transactions['is_return'] == 1].groupby('Product_ID')['Line_Total'].sum()
product_return_rate = (abs(product_returns) / product_gross * 100).round(2)

# Products with sales but zero returns
products_with_sales = product_gross.index
products_with_returns = product_returns.index
zero_return_products = [p for p in products_with_sales if p not in products_with_returns]

print(f"\n✅ PRODUCTS WITH ZERO RETURNS: {len(zero_return_products)}")
if len(zero_return_products) > 0:
    print("\nSample of products with zero returns (Top 10 by revenue):")
    zero_return_revenue = product_gross[zero_return_products].sort_values(ascending=False).head(10)
    for i, (prod_id, revenue) in enumerate(zero_return_revenue.items(), 1):
        name = product_names.get(prod_id, 'Unknown')[:50]
        print(f"   {i:2d}. {name:50s} ${revenue:>10,.2f}")

# Highest return rate products (with significant sales)
high_volume_products = product_gross[product_gross > product_gross.quantile(0.5)]
high_return_products = product_return_rate[product_return_rate.index.isin(high_volume_products.index)].sort_values(ascending=False).head(10)

print(f"\n❌ TOP 10 PRODUCTS WITH HIGHEST RETURN RATE (min revenue threshold):")
for i, (prod_id, rate) in enumerate(high_return_products.items(), 1):
    name = product_names.get(prod_id, 'Unknown')[:50]
    revenue = product_gross.get(prod_id, 0)
    returns = abs(product_returns.get(prod_id, 0))
    print(f"   {i:2d}. {name:50s}")
    print(f"       Return Rate: {rate:>6.2f}% | Revenue: ${revenue:>10,.2f} | Returns: ${returns:>10,.2f}")

# Visualization
fig = go.Figure()

# Show distribution of return rates
return_rate_dist = product_return_rate.value_counts(bins=20).sort_index()
fig.add_trace(go.Bar(
    x=[f'{i.left:.1f}-{i.right:.1f}%' for i in return_rate_dist.index],
    y=return_rate_dist.values,
    marker_color='lightcoral'
))

fig.update_layout(
    title='Distribution of Product Return Rates',
    xaxis_title='Return Rate Range',
    yaxis_title='Number of Products',
    height=400
)
fig.update_xaxes(tickangle=45)
fig.show()


B. RETURN ANALYSIS

💰 OVERALL RETURN METRICS:
   Gross Revenue:  $ 807,803,874.00
   Return Loss:    $ -45,324,477.54
   Net Revenue:    $ 762,479,396.46
   Return Rate:              5.61%

📦 B1. RETURN RATE BY CATEGORY

Return Rate by Category:
   Children  :   5.60%
   Feminine  :   5.65%
   Masculine :   5.57%



🌍 B2. RETURN RATE BY COUNTRY

Return Rate by Country:
   Portugal       :   5.82%
   United States  :   5.69%
   España         :   5.64%
   France         :   5.63%
   Deutschland    :   5.59%
   中国             :   5.59%
   United Kingdom :   5.54%



💸 B3. RETURN LOSS BY CATEGORY

Return Loss by Category:
   Children  : $   4,023,581.16
   Feminine  : $  21,132,040.45
   Masculine : $  20,168,855.93

🏆 B4. TOP 10 PRODUCTS BY NET REVENUE

Top 10 Products by Net Revenue:
    1. Male Fact With Built -In Pocket Detail             $  275,656.65
    2. Men'S Blazer With Clean Style And Contemporary Cut $  274,717.40
    3. Male Fact With Slim Fit Style And Built -In Belt D $  272,535.45
    4. Male Fact With Flanelado Fabric For Cold Climate   $  269,263.00
    5. Men'S Blazer Of Light Fabric With Cut In Thin Stri $  264,927.80
    6. Male Fact With Detail Of Leather Appliques         $  263,769.65
    7. Men'S Blazer With Zipper Details On The Sleeves    $  262,786.25
    8. Male Fact Of Casual Style Striped Twill            $  262,555.65
    9. Classic Linen Masculine For Summer                 $  254,003.60
   10. Masculine Linen Fact With Side Pocket Details      $  251,883.65

🏬 B5. RETURN RATE BY STORE

Top 10 Stores with Highest 


📊 B6. RETURN CORRELATION SUMMARY

✅ BEST STORE (Lowest Returns):
   Store: Store Birmingham
   Country: United Kingdom
   Return Rate: 5.38%

❌ WORST STORE (Highest Returns):
   Store: Store Coimbra
   Country: Portugal
   Return Rate: 6.12%

📈 RETURN RATE STATISTICS:
   Average Store Return Rate: 5.65%
   Median Store Return Rate:  5.64%
   Std Deviation:             0.15%
   Range:                     5.38% - 6.12%

📦 B7. PRODUCTS WITH LOWEST/HIGHEST RETURNS

✅ PRODUCTS WITH ZERO RETURNS: 88

Sample of products with zero returns (Top 10 by revenue):
    1. Slim Fit Men'S Linen Pants                         $ 22,744.85
    2. Plucked Children'S Sweatshirt With Tape Detail     $ 20,002.15
    3. Women'S Cigarette Pants                            $ 19,077.00
    4. Slim Fit Casual Men'S Pants                        $ 13,191.80
    5. Streetwear Smooth Beige Income                     $ 12,057.52
    6. Luxury Lilac Jacquard With Glitter                 $  9,232.28
    7. Smooth Edge Li

# C. PAYMENT METHOD ANALYSIS

In [13]:
print("\n" + "="*80)
print("C. PAYMENT METHOD ANALYSIS")
print("="*80)

print("\n💳 REVENUE BY PAYMENT METHOD")

payment_revenue = transactions.groupby('Payment_Method')['Line_Total'].sum().sort_values(ascending=False)

print("\nRevenue by Payment Method:")
for method, revenue in payment_revenue.items():
    share = revenue / net_revenue * 100
    print(f"   {method:15s}: ${revenue:>15,.2f} ({share:>5.2f}%)")

# Visualization
fig = go.Figure(data=[
    go.Bar(
        x=payment_revenue.index,
        y=payment_revenue.values,
        marker_color=['#3498db', '#2ecc71'],
        text=[f'${v/1e6:.1f}M' for v in payment_revenue.values],
        textposition='outside'
    )
])
fig.update_layout(
    title='Revenue by Payment Method',
    xaxis_title='Payment Method',
    yaxis_title='Revenue ($)',
    height=400
)
fig.show()


C. PAYMENT METHOD ANALYSIS

💳 REVENUE BY PAYMENT METHOD

Revenue by Payment Method:
   Credit Card    : $ 475,707,989.80 (62.39%)
   Cash           : $ 286,771,406.66 (37.61%)


# D. SEASONALITY & SALES TRENDS

In [14]:
print("\n" + "="*80)
print("D. SEASONALITY & SALES TRENDS")
print("="*80)

# --- D1. Monthly/Quarterly Trends ---
print("\n📈 D1. MONTHLY/QUARTERLY REVENUE TRENDS")

transactions['Quarter'] = transactions['Date'].dt.to_period('Q')
quarterly_revenue = transactions.groupby('Quarter')['Line_Total'].sum().reset_index()
quarterly_revenue['Quarter'] = quarterly_revenue['Quarter'].astype(str)

print("\nQuarterly Revenue (Last 8 quarters):")
for _, row in quarterly_revenue.tail(8).iterrows():
    print(f"   {row['Quarter']}: ${row['Line_Total']:>12,.2f}")

# Visualization
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=monthly_revenue['YearMonth'],
    y=monthly_revenue['Line_Total'],
    mode='lines+markers',
    name='Monthly',
    line=dict(width=2)
))
fig.update_layout(
    title='Monthly Revenue Trend with Seasonality',
    xaxis_title='Month',
    yaxis_title='Revenue ($)',
    height=400
)
fig.update_xaxes(tickangle=45)
fig.show()

# --- D2. Peak Sales Days ---
print("\n📅 D2. PEAK SALES DAYS")

transactions['DayOfWeek'] = transactions['Date'].dt.day_name()
day_revenue = transactions.groupby('DayOfWeek')['Line_Total'].sum()

# Order days correctly
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_revenue = day_revenue.reindex(day_order)

print("\nRevenue by Day of Week:")
for day, revenue in day_revenue.items():
    print(f"   {day:10s}: ${revenue:>15,.2f}")

# Visualization
fig = go.Figure(data=[
    go.Bar(
        x=day_revenue.index,
        y=day_revenue.values,
        marker_color='lightgreen',
        text=[f'${v/1e6:.1f}M' for v in day_revenue.values],
        textposition='outside'
    )
])
fig.update_layout(
    title='Revenue by Day of Week',
    xaxis_title='Day',
    yaxis_title='Revenue ($)',
    height=400
)
fig.show()

# --- D3. Year-over-Year Comparison ---
print("\n📊 D3. YEAR-OVER-YEAR COMPARISON")

transactions['Year'] = transactions['Date'].dt.year
transactions['Month'] = transactions['Date'].dt.month

yearly_monthly = transactions.groupby(['Year', 'Month'])['Line_Total'].sum().reset_index()

print("\nYear-over-Year Revenue by Month:")
for year in sorted(yearly_monthly['Year'].unique()):
    year_data = yearly_monthly[yearly_monthly['Year'] == year]
    total = year_data['Line_Total'].sum()
    print(f"   {year}: ${total:>15,.2f}")

# Visualization
fig = go.Figure()
for year in sorted(yearly_monthly['Year'].unique()):
    year_data = yearly_monthly[yearly_monthly['Year'] == year]
    fig.add_trace(go.Scatter(
        x=year_data['Month'],
        y=year_data['Line_Total'],
        mode='lines+markers',
        name=str(year),
        line=dict(width=2)
    ))
fig.update_layout(
    title='Year-over-Year Monthly Revenue Comparison',
    xaxis_title='Month',
    yaxis_title='Revenue ($)',
    height=400
)
fig.show()


D. SEASONALITY & SALES TRENDS

📈 D1. MONTHLY/QUARTERLY REVENUE TRENDS

Quarterly Revenue (Last 8 quarters):
   2023Q2: $76,781,711.02
   2023Q3: $79,693,809.42
   2023Q4: $137,531,660.47
   2024Q1: $68,583,067.71
   2024Q2: $80,536,354.78
   2024Q3: $84,374,623.44
   2024Q4: $138,366,255.33
   2025Q1: $26,777,992.33



📅 D2. PEAK SALES DAYS

Revenue by Day of Week:
   Monday    : $  88,101,198.46
   Tuesday   : $  85,411,874.79
   Wednesday : $  85,732,367.14
   Thursday  : $  83,905,604.47
   Friday    : $  83,268,906.86
   Saturday  : $ 164,644,851.81
   Sunday    : $ 171,414,592.93



📊 D3. YEAR-OVER-YEAR COMPARISON

Year-over-Year Revenue by Month:
   2023: $ 363,841,102.87
   2024: $ 371,860,301.26
   2025: $  26,777,992.33


# E. CUSTOMER METRICS

In [15]:
print("\n" + "="*80)
print("E. CUSTOMER METRICS")
print("="*80)

# Customer-level aggregation
customer_metrics = trans_country.groupby(['Customer_ID', 'Country']).agg({
    'Invoice_ID': 'nunique',
    'Line_Total': 'sum'
}).reset_index()
customer_metrics.columns = ['Customer_ID', 'Country', 'total_orders', 'net_spend']

# Total customers
total_customers = customer_metrics['Customer_ID'].nunique()

# Repeat vs one-time
repeat_customers = customer_metrics[customer_metrics['total_orders'] > 1]
onetime_customers = customer_metrics[customer_metrics['total_orders'] == 1]

repeat_count = len(repeat_customers)
onetime_count = len(onetime_customers)
repeat_rate = repeat_count / total_customers * 100

# Average customer value
avg_customer_value = customer_metrics['net_spend'].mean()

print(f"\n👥 CUSTOMER OVERVIEW:")
print(f"   Total Customers:         {total_customers:>10,}")
print(f"   Repeat Customers:        {repeat_count:>10,}")
print(f"   One-time Customers:      {onetime_count:>10,}")
print(f"   Repeat Customer Rate:    {repeat_rate:>13.2f}%")
print(f"   Average Customer Value:  ${avg_customer_value:>14,.2f}")

# CLV by country
clv_by_country = customer_metrics.groupby('Country')['net_spend'].mean().sort_values(ascending=False)

print(f"\n💰 CUSTOMER LIFETIME VALUE (CLV) BY COUNTRY:")
print("   (CLV = Total Net Spend per Customer)")
for country, clv in clv_by_country.items():
    print(f"   {country:15s}: ${clv:>10,.2f}")

# Visualization
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Customer Type Distribution', 'Avg CLV by Country'),
    specs=[[{'type': 'domain'}, {'type': 'xy'}]]
)

# Pie chart
fig.add_trace(
    go.Pie(
        labels=['Repeat', 'One-time'],
        values=[repeat_count, onetime_count],
        hole=0.4,
        marker_colors=['#2ecc71', '#e74c3c']
    ),
    row=1, col=1
)

# Bar chart
fig.add_trace(
    go.Bar(
        x=clv_by_country.index,
        y=clv_by_country.values,
        marker_color='lightblue'
    ),
    row=1, col=2
)

fig.update_layout(title='Customer Metrics Analysis', height=400)
fig.show()


E. CUSTOMER METRICS

👥 CUSTOMER OVERVIEW:
   Total Customers:          1,268,571
   Repeat Customers:           943,304
   One-time Customers:         325,267
   Repeat Customer Rate:            74.36%
   Average Customer Value:  $        601.05

💰 CUSTOMER LIFETIME VALUE (CLV) BY COUNTRY:
   (CLV = Total Net Spend per Customer)
   中国             : $  1,992.78
   United States  : $    242.33
   Portugal       : $    202.39
   Deutschland    : $    180.95
   France         : $    164.92
   España         : $    157.41
   United Kingdom : $    133.25


# F. DISCOUNT & PROMOTION METRICS

In [16]:
print("\n" + "="*80)
print("F. DISCOUNT & PROMOTION METRICS")
print("="*80)

# Use only positive transactions
campaign_trans = trans_positive[trans_positive['has_discount'] == 1]

revenue_during_campaign = campaign_trans['Line_Total'].sum()
discount_amount = campaign_trans['Discount'].sum()
pre_discount_revenue = revenue_during_campaign + discount_amount

campaign_transactions = len(campaign_trans)
total_positive_trans = len(trans_positive)
discount_penetration = campaign_transactions / total_positive_trans * 100

# Discount ROI
discount_roi = revenue_during_campaign / discount_amount if discount_amount > 0 else 0

print(f"\n🏷️ DISCOUNT CAMPAIGN OVERVIEW:")
print(f"   Revenue During Campaign:   ${revenue_during_campaign:>15,.2f}")
print(f"   Pre-Discount Revenue:      ${pre_discount_revenue:>15,.2f}")
print(f"   Total Discount Amount:     ${discount_amount:>15,.2f}")
print(f"   Discount ROI:              {discount_roi:>18.2f}x")
print(f"   Discount Penetration:      {discount_penetration:>17.2f}%")

# Category-level discount analysis
campaign_category = campaign_trans.merge(products[['Product_ID', 'Category']], on='Product_ID')
category_discount = campaign_category.groupby('Category').agg({
    'Line_Total': 'sum',
    'Discount': ['sum', 'mean']
}).round(2)
category_discount.columns = ['revenue', 'total_discount', 'avg_discount']

print(f"\n📊 CATEGORY-LEVEL DISCOUNT ANALYSIS:")
for category in category_discount.index:
    row = category_discount.loc[category]
    roi = row['revenue'] / row['total_discount'] if row['total_discount'] > 0 else 0
    print(f"   {category:10s}: Revenue ${row['revenue']:>12,.2f} | "
          f"Discount ${row['total_discount']:>10,.2f} | ROI {roi:.2f}x")

# Visualization
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Discount Penetration', 'Discount Amount by Category'),
    specs=[[{'type': 'domain'}, {'type': 'xy'}]]
)

# Penetration
fig.add_trace(
    go.Pie(
        labels=['Campaign', 'Regular'],
        values=[campaign_transactions, total_positive_trans - campaign_transactions],
        hole=0.4
    ),
    row=1, col=1
)

# Category
fig.add_trace(
    go.Bar(
        x=category_discount.index,
        y=category_discount['total_discount'],
        marker_color='coral'
    ),
    row=1, col=2
)

fig.update_layout(title='Discount Campaign Analysis', height=400)
fig.show()


F. DISCOUNT & PROMOTION METRICS

🏷️ DISCOUNT CAMPAIGN OVERVIEW:
   Revenue During Campaign:   $ 153,576,542.75
   Pre-Discount Revenue:      $ 154,342,489.25
   Total Discount Amount:     $     765,946.50
   Discount ROI:                          200.51x
   Discount Penetration:                  30.02%

📊 CATEGORY-LEVEL DISCOUNT ANALYSIS:
   Children  : Revenue $25,448,753.22 | Discount $127,055.40 | ROI 200.30x
   Feminine  : Revenue $65,300,434.45 | Discount $344,476.45 | ROI 189.56x
   Masculine : Revenue $62,827,355.08 | Discount $294,414.65 | ROI 213.40x


# G. CUSTOMER SEGMENTATION

In [9]:
print("\n" + "="*80)
print("G. CUSTOMER SEGMENTATION (RFM ANALYSIS)")
print("="*80)

# Prepare data for segmentation
from datetime import datetime

# Use only positive transactions for customer behavior
customer_trans = trans_country[trans_country['is_return'] == 0].copy()

# Calculate RFM metrics
current_date = customer_trans['Date'].max()

rfm = customer_trans.groupby('Customer_ID').agg({
    'Date': lambda x: (current_date - x.max()).days,  # Recency
    'Invoice_ID': 'nunique',  # Frequency
    'Line_Total': 'sum'  # Monetary
}).reset_index()

rfm.columns = ['Customer_ID', 'Recency', 'Frequency', 'Monetary']

print(f"\n📊 RFM METRICS CALCULATED:")
print(f"   Total customers analyzed: {len(rfm):,}")

# Create RFM scores (1-5 scale)
rfm['R_Score'] = pd.qcut(rfm['Recency'], q=5, labels=[5,4,3,2,1], duplicates='drop')
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), q=5, labels=[1,2,3,4,5], duplicates='drop')
rfm['M_Score'] = pd.qcut(rfm['Monetary'].rank(method='first'), q=5, labels=[1,2,3,4,5], duplicates='drop')

# Create RFM segment
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)
rfm['RFM_Total'] = rfm['R_Score'].astype(int) + rfm['F_Score'].astype(int) + rfm['M_Score'].astype(int)

# Segment customers
def segment_customer(row):
    if row['RFM_Total'] >= 13:
        return 'Champions'
    elif row['RFM_Total'] >= 11:
        return 'Loyal Customers'
    elif row['RFM_Total'] >= 9:
        return 'Potential Loyalists'
    elif row['RFM_Total'] >= 7:
        return 'At Risk'
    elif row['RFM_Total'] >= 5:
        return 'Need Attention'
    else:
        return 'Lost'

rfm['Segment'] = rfm.apply(segment_customer, axis=1)

# Segment analysis
segment_summary = rfm.groupby('Segment').agg({
    'Customer_ID': 'count',
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': ['mean', 'sum']
}).round(2)

segment_summary.columns = ['Customer_Count', 'Avg_Recency', 'Avg_Frequency', 'Avg_Monetary', 'Total_Revenue']
segment_summary = segment_summary.sort_values('Total_Revenue', ascending=False)

print(f"\n👥 CUSTOMER SEGMENTS:")
print(f"\n{'Segment':<20} {'Customers':>12} {'Avg Recency':>12} {'Avg Freq':>10} {'Avg Spend':>12} {'Total Revenue':>15}")
print("-" * 90)
for segment in segment_summary.index:
    row = segment_summary.loc[segment]
    print(f"{segment:<20} {row['Customer_Count']:>12,.0f} {row['Avg_Recency']:>12.0f} {row['Avg_Frequency']:>10.1f} ${row['Avg_Monetary']:>11,.2f} ${row['Total_Revenue']:>14,.2f}")

# Calculate segment value
segment_summary['Revenue_Share'] = (segment_summary['Total_Revenue'] / segment_summary['Total_Revenue'].sum() * 100).round(2)
segment_summary['Customer_Share'] = (segment_summary['Customer_Count'] / segment_summary['Customer_Count'].sum() * 100).round(2)

print(f"\n📊 SEGMENT VALUE CONTRIBUTION:")
for segment in segment_summary.index:
    row = segment_summary.loc[segment]
    print(f"   {segment:<20}: {row['Customer_Share']:>5.2f}% of customers → {row['Revenue_Share']:>5.2f}% of revenue")

# Visualizations
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Customer Distribution by Segment',
        'Revenue Distribution by Segment',
        'Avg Customer Value by Segment',
        'RFM Scores Distribution'
    ),
    specs=[
        [{'type': 'pie'}, {'type': 'pie'}],
        [{'type': 'bar'}, {'type': 'scatter'}]
    ]
)

# Customer count pie
fig.add_trace(
    go.Pie(
        labels=segment_summary.index,
        values=segment_summary['Customer_Count'],
        hole=0.4
    ),
    row=1, col=1
)

# Revenue pie
fig.add_trace(
    go.Pie(
        labels=segment_summary.index,
        values=segment_summary['Total_Revenue'],
        hole=0.4
    ),
    row=1, col=2
)

# Avg value bar
fig.add_trace(
    go.Bar(
        x=segment_summary.index,
        y=segment_summary['Avg_Monetary'],
        marker_color='lightblue',
        text=[f'${v:,.0f}' for v in segment_summary['Avg_Monetary']],
        textposition='outside'
    ),
    row=2, col=1
)

# RFM scatter (Recency vs Monetary)
segment_colors = {
    'Champions': 'gold',
    'Loyal Customers': 'green',
    'Potential Loyalists': 'blue',
    'At Risk': 'orange',
    'Need Attention': 'red',
    'Lost': 'gray'
}

for segment in rfm['Segment'].unique():
    segment_data = rfm[rfm['Segment'] == segment].sample(min(100, len(rfm[rfm['Segment'] == segment])))
    fig.add_trace(
        go.Scatter(
            x=segment_data['Recency'],
            y=segment_data['Monetary'],
            mode='markers',
            name=segment,
            marker=dict(
                size=8,
                color=segment_colors.get(segment, 'blue'),
                opacity=0.6
            )
        ),
        row=2, col=2
    )

fig.update_xaxes(title_text="Segment", row=2, col=1)
fig.update_yaxes(title_text="Avg Monetary ($)", row=2, col=1)
fig.update_xaxes(title_text="Recency (days)", row=2, col=2)
fig.update_yaxes(title_text="Monetary ($)", row=2, col=2)

fig.update_layout(
    title_text='Customer Segmentation Analysis (RFM)',
    showlegend=True,
    height=800
)
fig.show()

# Strategic recommendations by segment
print(f"\n💡 STRATEGIC RECOMMENDATIONS BY SEGMENT:")

recommendations = {
    'Champions': 'Reward with VIP benefits, early access, exclusive offers. Potential brand ambassadors.',
    'Loyal Customers': 'Upsell premium products, loyalty programs, referral incentives.',
    'Potential Loyalists': 'Engagement campaigns, personalized offers, increase purchase frequency.',
    'At Risk': 'Win-back campaigns, surveys to understand issues, special discounts.',
    'Need Attention': 'Reactivation emails, limited-time offers, remind of product benefits.',
    'Lost': 'Aggressive win-back with deep discounts, understand why they left.'
}

for segment, recommendation in recommendations.items():
    if segment in segment_summary.index:
        count = segment_summary.loc[segment, 'Customer_Count']
        revenue = segment_summary.loc[segment, 'Total_Revenue']
        print(f"\n   {segment} ({count:,.0f} customers, ${revenue:,.2f} revenue):")
        print(f"   → {recommendation}")


G. CUSTOMER SEGMENTATION (RFM ANALYSIS)

📊 RFM METRICS CALCULATED:
   Total customers analyzed: 1,268,571

👥 CUSTOMER SEGMENTS:

Segment                 Customers  Avg Recency   Avg Freq    Avg Spend   Total Revenue
------------------------------------------------------------------------------------------
Champions                 235,579           54        7.2 $   1,823.95 $429,685,386.31
Loyal Customers           231,373          108        4.3 $     802.71 $185,726,504.67
Potential Loyalists       233,981          165        2.9 $     454.51 $106,346,246.74
At Risk                   230,092          240        2.0 $     254.69 $ 58,602,522.02
Need Attention            191,876          347        1.4 $     110.64 $ 21,229,916.50
Lost                      145,670          505        1.0 $      42.65 $  6,213,297.76

📊 SEGMENT VALUE CONTRIBUTION:
   Champions           : 18.57% of customers → 53.19% of revenue
   Loyal Customers     : 18.24% of customers → 22.99% of revenue
   Potent


💡 STRATEGIC RECOMMENDATIONS BY SEGMENT:

   Champions (235,579 customers, $429,685,386.31 revenue):
   → Reward with VIP benefits, early access, exclusive offers. Potential brand ambassadors.

   Loyal Customers (231,373 customers, $185,726,504.67 revenue):
   → Upsell premium products, loyalty programs, referral incentives.

   Potential Loyalists (233,981 customers, $106,346,246.74 revenue):
   → Engagement campaigns, personalized offers, increase purchase frequency.

   At Risk (230,092 customers, $58,602,522.02 revenue):
   → Win-back campaigns, surveys to understand issues, special discounts.

   Need Attention (191,876 customers, $21,229,916.50 revenue):
   → Reactivation emails, limited-time offers, remind of product benefits.

   Lost (145,670 customers, $6,213,297.76 revenue):
   → Aggressive win-back with deep discounts, understand why they left.


In [12]:
# Save RFM metrics to CSV
rfm.to_csv('customer_rfm_metrics.csv', index=False)
print("Customer RFM metrics saved to 'customer_rfm_metrics.csv'")

Customer RFM metrics saved to 'customer_rfm_metrics.csv'


#H.EMPLOYEE PRODUCTIVITY

In [10]:
print("\n" + "=" * 80)
print(" EMPLOYEE PERFORMANCE")
print("=" * 80)

# Employee revenue
emp_perf = transactions.groupby('Employee_ID').agg({
    'Line_Total': 'sum',
    'Invoice_ID': 'nunique',
    'Customer_ID': 'nunique'
}).round(2)
emp_perf.columns = ['revenue', 'orders', 'customers']

# Merge with employee details
emp_analysis = employees.merge(emp_perf.reset_index(), on='Employee_ID')
emp_analysis = emp_analysis.sort_values('revenue', ascending=False)

print(f"\n🏆 TOP 10 PERFORMERS:")
for idx, (_, emp) in enumerate(emp_analysis.head(10).iterrows(), 1):
    print(f"   {idx}. {emp['Name']}: ${emp['revenue']:,.2f}")

print(f"\n⚠️ BOTTOM 10 PERFORMERS:")
for idx, (_, emp) in enumerate(emp_analysis.tail(10).iterrows(), 1):
    print(f"   {idx}. {emp['Name']}: ${emp['revenue']:,.2f}")

# Productivity gap analysis
top_performer_revenue = emp_analysis['revenue'].max()
bottom_performer_revenue = emp_analysis['revenue'].min()
productivity_gap = top_performer_revenue / bottom_performer_revenue

print(f"\n📊 PRODUCTIVITY METRICS:")
print(f"   Avg Revenue per Employee: ${emp_analysis['revenue'].mean():,.2f}")
print(f"   Top Performer: ${top_performer_revenue:,.2f}")
print(f"   Bottom Performer: ${bottom_performer_revenue:,.2f}")
print(f"   Productivity Gap: {productivity_gap:.1f}x")

# Check if gap is geography-based
emp_with_stores = emp_analysis.merge(stores[['Store_ID', 'Country']], on='Store_ID')
country_emp_avg = emp_with_stores.groupby('Country')['revenue'].mean().sort_values(ascending=False)

print(f"\n🌍 AVG EMPLOYEE REVENUE BY COUNTRY:")
for country, avg_rev in country_emp_avg.items():
    print(f"   {country}: ${avg_rev:,.2f}")


 EMPLOYEE PERFORMANCE

🏆 TOP 10 PERFORMERS:
   1. 韩娜: $20,287,147.48
   2. 孙桂芳: $20,237,872.71
   3. 张淑英: $20,231,963.81
   4. 张林: $20,195,951.13
   5. 章帆: $20,112,267.80
   6. 周梅: $20,036,627.49
   7. 张玉华: $19,986,079.71
   8. 窦涛: $19,984,973.55
   9. 谢兰英: $19,936,600.65
   10. 阎莹: $15,523,513.81

⚠️ BOTTOM 10 PERFORMERS:
   1. Martin Morgan: $334,407.72
   2. Brittany Ramos: $262,294.63
   3. Joshua Brown: $261,200.14
   4. Andre Page: $260,475.95
   5. Brittany Wang: $259,893.92
   6. Antonio Smith: $256,347.52
   7. Maria Brown: $255,795.15
   8. Todd Buchanan: $254,920.41
   9. Mary Arnold: $254,443.27
   10. Kevin Rogers: $254,259.88

📊 PRODUCTIVITY METRICS:
   Avg Revenue per Employee: $2,899,161.20
   Top Performer: $20,287,147.48
   Bottom Performer: $254,259.88
   Productivity Gap: 79.8x

🌍 AVG EMPLOYEE REVENUE BY COUNTRY:
   中国: $14,501,014.87
   United States: $2,207,881.77
   Deutschland: $671,390.27
   France: $659,031.11
   España: $631,843.65
   United Kingdom: $487,04